In [1]:
from dotenv import load_dotenv
from dotenv import dotenv_values

In [10]:
FILE_PATH = "C:/Ambarish/NCERT/"

In [11]:
load_dotenv()
values_env = dotenv_values(".env")
URL = values_env['URL']
COLLECTION_NAME = values_env['COLLECTION_NAME']
DIMENSION = int(values_env['DIMENSION'])
MODEL_NAME = values_env['MODEL_NAME']

In [12]:
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer

import qdrant_client as qc
import qdrant_client.http.models as qmodels
from qdrant_client.http.models import *

import os
import uuid

In [13]:
# Read the PDF file and return the text
def get_pdf_data(file_path, num_pages = 1):
    reader = PdfReader(file_path)
    full_doc_text = ""
    pages = reader.pages
    num_pages = len(pages) 
    
    try:
        for page in range(num_pages):
            current_page = reader.pages[page]
            text = current_page.extract_text()
            full_doc_text += text
    except:
        print("Error reading file")
    finally:
        return full_doc_text

In [14]:
# Divide the text into chunks of chunk_length 
# [ default is 500] characters
def get_chunks(fulltext:str,chunk_length =500) -> list:
    text = fulltext

    chunks = []
    while len(text) > chunk_length:
        last_period_index = text[:chunk_length].rfind('.')
        if last_period_index == -1:
            last_period_index = chunk_length
        chunks.append(text[:last_period_index])
        text = text[last_period_index+1:]
    chunks.append(text)

    return chunks

In [15]:
model = SentenceTransformer(MODEL_NAME)

In [16]:
client = qc.QdrantClient(url=URL)
METRIC = qmodels.Distance.COSINE

In [17]:
# Create embeddings for the chunks
# Insert the chunks into the Qdrant collection
# Insert the metadata for the chunks into the Qdrant collection
FILES = os.listdir(FILE_PATH)
FILES_FULL_PATH = [FILE_PATH + file for file in FILES]
for filename in FILES_FULL_PATH:
    print(f'Processing file: {filename}')
    full_doc_text = get_pdf_data(filename)
    print(f'Full doc text length: {len(full_doc_text)}')
    payloads = []
    li_id = []
    corpus = []
    Lines =get_chunks(full_doc_text,500)
    for token in Lines:
        corpus.append(token)
        payloads.append({"token":token,
                         "filename": os.path.basename(filename),
                           "type":"pdf"})
        li_id.append(str(uuid.uuid4()))
    embeddings_all = model.encode(corpus, convert_to_tensor=True)
    print(f'Full embeddings length: {len(embeddings_all)}')

    CHUNK_SIZE = 100
    for i in range(0, len(embeddings_all), CHUNK_SIZE):
        if(i+CHUNK_SIZE > len(embeddings_all) -1):
            new_chunk = len(embeddings_all) -1
        else:
            new_chunk = i+CHUNK_SIZE -1
        print("Inserting chunk", i , "to", new_chunk)
        client.upsert(
            collection_name=COLLECTION_NAME,
            points=qmodels.Batch(
                ids = li_id[i:new_chunk],
                vectors=embeddings_all[i:new_chunk].tolist(),
                payloads=payloads[i:new_chunk]
            ),
        )

Processing file: C:/Ambarish/NCERT/BIOTECH.pdf
Full doc text length: 25081
Full embeddings length: 60
Inserting chunk 0 to 59
Processing file: C:/Ambarish/NCERT/CHAP04-BIOLOGY-CLASS11.pdf
Full doc text length: 31802
Full embeddings length: 73
Inserting chunk 0 to 72
Processing file: C:/Ambarish/NCERT/HUMAN-WELFARE.pdf
Full doc text length: 51060
Full embeddings length: 121
Inserting chunk 0 to 99
Inserting chunk 100 to 120
